In [ ]:
import pyspark
import pyspark.sql.functions as f
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql.functions import sum as _sum
import math

In [ ]:
df = spark.read.text(path)

In [ ]:
df_split = df.select(f.split(df.value," ")).rdd.flatMap(lambda x: x).toDF(schema=["command","value"])

In [ ]:
def metres_counter(first_point: str, second_point: str, df: pyspark.sql.dataframe) -> pyspark.sql.dataframe:
    df = df.filter(df.command != "picture")
    df = df.select("*").withColumn("id", monotonically_increasing_id())
    X = df.filter(df.command == "marker").filter(df.value == f"{first_point}").select('id').collect()[0][0]
    Y = df.filter(df.command == "marker").filter(df.value == f"{second_point}").select('id').collect()[0][0]
    df_slice = spark.createDataFrame(df.collect()[X:Y])
    df_final = df_slice.filter(df_slice.command == "forward")
    
    return df_final.agg(_sum('value').alias('sum_metres'))
    
    

In [ ]:
df_test = spark.read.text(path)
df_split_test = df_test.select(f.split(df_test.value," ")).rdd.flatMap(lambda x: x).toDF(schema=["command","value"])

In [ ]:
sum_val = metres_counter('A', 'C', df_split_test)

In [ ]:
sum_val.show()

+----------+
sum_metres|
+----------+
 3886.0|
+----------+

In [ ]:
def change_the_sign(x) -> pyspark.rdd.PipelinedRDD:
    id_val = x.id
    command = x.command
    if x.command == 'left':
        x_val = (-1) * int(x.value)
    else:
        x_val = int(x.value)
    
    return (id_val, command, x_val)

In [ ]:
def slice_maker(first_point: str, second_point: str, df: pyspark.sql.dataframe) -> pyspark.sql.dataframe:
    df = df.filter(df.command != "picture")
    df = df.select("*").withColumn("id", monotonically_increasing_id())
    X = df.filter(df.command == "marker").filter(df.value == f"{first_point}").select('id').collect()[0][0]
    Y = df.filter(df.command == "marker").filter(df.value == f"{second_point}").select('id').collect()[0][0]
    df = spark.createDataFrame(df.collect()[X:Y])
    df_final = df.filter(df.command != "marker")
    
    return df_final

In [ ]:
def split_the_df(df: pyspark.sql.dataframe) -> pyspark.sql.dataframe:
    a = []
    angle = []
    data_collect = df.collect()
    if data_collect[-1]['command'] == 'right' or data_collect[-1]['command'] == 'left':
        data_collect.pop(-1)
    else:
        pass
    for i in range(len(data_collect)):
        if data_collect[i]['command'] == 'forward':
            if data_collect[i]['command'] == data_collect[i-1]['command']:
                a.append(data_collect[i]['value'])
                angle.append(0)
            else:
                a.append(data_collect[i]['value'])
        else:
            angle.append(data_collect[i]['value'])

    data = [x for x in zip(a, angle)]

    return spark.createDataFrame(data=data, schema = ['distance', 'angle'])

In [ ]:
def find_point_coordinates(df: pyspark.sql.dataframe) -> list:
    angle = 0
    x = 0
    y = 0
    data_collect = df.collect()
    for i in range(len(data_collect)):
        d = data_collect[i]['distance']
        angle = data_collect[i]['angle'] + angle
        x = x + (d * math.cos(math.radians(angle)))
        y = y + (d * math.sin(math.radians(angle)))
        
    return [x, y]

In [ ]:
def count_straight_line(first_point: str, second_point: str, df: pyspark.sql.dataframe) -> float:
    points = [first_point, second_point]
    points_coordinates = {}
    for point in points:
        if point == 'Origin':
            points_coordinates[point] = [0, 0]
        else:
            df_slice = slice_maker('Origin', point, df)
            rdd = df_slice.rdd.map(lambda x: change_the_sign(x))
            df_slice = rdd.toDF(schema=['id', 'command', 'value'])
            df_splitted = split_the_df(df_slice)

            points_coordinates[point] = find_point_coordinates(df_splitted)
    
    return math.dist(points_coordinates[first_point], points_coordinates[second_point])

In [ ]:
distance = count_straight_line('A', 'C', df_split_test)

In [ ]:
print(f'Distance between two given points is {round(distance, 2)} metres')

Distance between two given points is 709.85 metres

In [ ]:
distance = count_straight_line('Origin', 'C', df_split_test)

In [ ]:
print(f'Distance between two given points is {round(distance, 2)} metres')

Distance between two given points is 1023.78 metres